In [1]:
%%writefile ed_tools.py
# -*- coding: utf-8 -*-
from __future__ import print_function, division, absolute_import, unicode_literals

import numba
import torch as th
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix

@numba.jit(parallel=True)
def diagonalize_hamiltonian(H, blocks):
    '''...'''
    U_full = csr_matrix(H.shape, dtype=np.float32)
    E_full = np.zeros(H.shape[0])
    for i in numba.prange(len(blocks)):
        block = blocks[i]
        X,Y = np.meshgrid(block,block)
        E, U = np.linalg.eigh(H[X,Y].todense())
        E_full[block] = E
        U_full[Y,X] = U

    E = np.array(E_full)
    E0 = np.min(E)
    E = E - E0
    return E, E0, U_full

def get_frequency_greens_function_component(op2eb, iwn, op1, op2, beta, E, U, Z, xi=-1.0):

    r"""
    Returns:
    G^{(2)}(i\omega_n) = -1/Z < O_1(i\omega_n) O_2(-i\omega_n) >
    """
    op1_eig, op2_eig = op2eb(U, [op1.astype(np.float32), op2.astype(np.float32)])
    # -- Compute Lehman sum for all operator combinations
    G = np.zeros((len(iwn)), dtype=np.complex64)
    op = (op1_eig.getH().multiply(op2_eig)).tocoo().astype(np.float32)
    del op1_eig, op2_eig
    M = (np.exp(-beta * E[op.row]) - xi * np.exp(-beta * E[op.col])) * op.data
    e = (E[op.row] - E[op.col])
    del op
    for i in range(len(iwn)):
        G[i] = np.sum(M / (iwn[i] - e))
    G /= Z
    return G


def calculate_partition_function(beta, E):
    return np.sum(np.exp(-beta * E))

def operators_to_eigenbasis_cpu(U, op_vec):
    dop_vec = []
    u = U.toarray()
    for op in op_vec:
        M = u.T.dot(op.toarray().dot(u))
        dop_vec.append(coo_matrix(M, dtype=np.float32))

    return dop_vec

def operators_to_eigenbasis_gpu(U, op_vec):
    dop_vec = []
    d = th.FloatTensor(U.data)
    rc = th.LongTensor(np.vstack((U.row, U.col)))
    u = th.sparse.FloatTensor(rc, d,  th.Size(U.shape))
    del d, rc
    th.cuda.empty_cache()
    with th.cuda.device(0):
        u = u.cuda()
    for op in op_vec:
        with th.cuda.device(0):
            data = op.data.astype(np.float32)
            d = th.FloatTensor(data)
            rc = th.LongTensor(np.vstack((op.row, op.col)))
            tm = th.sparse.FloatTensor(rc, d,  th.Size(op.shape)).cuda()

            del d, rc
            th.cuda.empty_cache()

            temp = th.spmm(tm, u.to_dense())

            del tm
            th.cuda.empty_cache()

            dop_gpu = th.spmm(u.t(), temp)

            del temp
            th.cuda.empty_cache()

            dop = csr_matrix(dop_gpu.cpu().numpy(), dtype=np.float32)

            del dop_gpu
            th.cuda.empty_cache()
        dop_vec.append(dop)
    del u
    th.cuda.empty_cache()
    return dop_vec

Overwriting ed_tools.py


In [24]:
import anderson_model as am

In [25]:
beta = 10.0
num_iw = 64
wn = np.pi * (2 * np.array(range(-int(num_iw / 2), num_iw / 2)) + 1) / beta

In [26]:
H = am.A(mc = 2,
        mu = 0.15625,
        n_max = 80,
        Ud = 0., 
        Vcd = [2, -2],
        ec = [-1, 1],
        gamma_bd = 0.23, 
        eb = 0.07, 
        delta = 1.0, calc = True)

In [27]:
e, e0, u = diagonalize_hamiltonian(H.get_H(), H.get_blocks())

/home/aegaeon/anaconda3/envs/aegaeon/lib/python2.7/site-packages/ipykernel_launcher.py:1: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  """Entry point for launching an IPython kernel.


In [30]:
z = calculate_partition_function(beta, e)

op1, op2 = H.c_dag_up(), H.c_up()

gf = get_frequency_greens_function_component(operators_to_eigenbasis_cpu, 1j * wn, op1, op2, beta, e, u, z)

In [31]:
gf

array([-0.00061502+0.04947894j, -0.00065419+0.05103101j,
       -0.00069716+0.05268105j, -0.00074442+0.05443838j,
       -0.00079656+0.05631346j, -0.00085424+0.05831809j,
       -0.00091828+0.06046564j, -0.00098959+0.0627713j ,
       -0.0010693 +0.06525232j, -0.00115874+0.06792844j,
       -0.00125948+0.07082222j, -0.00137343+0.07395951j,
       -0.00150288+0.07737004j, -0.00165061+0.08108792j,
       -0.00182002+0.08515242j, -0.00201521+0.08960849j,
       -0.00224122+0.09450743j, -0.00250419+0.09990713j,
       -0.00281152+0.10587182j, -0.00317212+0.1124704j ,
       -0.00359632+0.1197724j , -0.00409565+0.12783895j,
       -0.00468155+0.13670495j, -0.00536258+0.14634483j,
       -0.00613804+0.15661004j, -0.00698559+0.16712102j,
       -0.00784042+0.17710233j, -0.00857043+0.18520932j,
       -0.00898685+0.18968308j, -0.00904945+0.19035073j,
       -0.00999505+0.20012471j, -0.0334888 +0.36895093j,
       -0.0334888 -0.36895093j, -0.00999505-0.20012471j,
       -0.00904945-0.19035073j,